In [1]:
# Import libraries
import ee
import folium
import geemap
import matplotlib.pyplot as plt


In [2]:
# Initialize the Earth Engine API
ee.Authenticate()
ee.Initialize(project = "class2-449500")


In [3]:
# Load protected area
yanachaga_geom = ee.FeatureCollection("projects/e115-jgarciayi/assets/YanachagaChemillen").geometry().dissolve(maxError=1)
center = yanachaga_geom.centroid().coordinates().getInfo()[::-1]
center[1] += 0.2  # Shift map center slightly east (moves map content left)

# Define bird species with full names and asset paths
#Tinamus osgoodi not available
bird_layers = {
    "Andigena hypoglauca": "projects/e115-jgarciayi/assets/CoordinatesAndigenaOnlyNP",
#    "Aulacorhynchus coeruleicinctis": "projects/e115-jgarciayi/assets/CoordinatesAulacorhynchusOnlyNP",
    "Doliornis sclateri": "projects/e115-jgarciayi/assets/CoordinatesDoliornisOnlyNP",
    "Gallinago jamesoni": "projects/e115-jgarciayi/assets/CoordinatesGallinagoOnlyNP",
    "Hapalopsittaca melanotis": "projects/e115-jgarciayi/assets/CoordinatesHapalopsittacaOnlyNP",
    "Pionus tumultuosus": "projects/e115-jgarciayi/assets/CoordinatesPionusTumultuosusOnlyNP",
    "Pipile cumanensis": "projects/e115-jgarciayi/assets/CoordinatesPipileCumanensisOnlyNP",
    "Rupicola peruvianus": "projects/e115-jgarciayi/assets/CoordinatesRupicolaPeruvianusOnlyNP"
}

# Assign unique colors
bird_colors = {
    "Andigena hypoglauca": "e41a1c",
#    "Aulacorhynchus coeruleicinctis": "377eb8",
    "Doliornis sclateri": "4daf4a",
    "Gallinago jamesoni": "984ea3",
    "Hapalopsittaca melanotis": "ff7f00",
    "Pionus tumultuosus": "a65628",
    "Pipile cumanensis": "f781bf",
    "Rupicola peruvianus": "999999"
}

# Create folium map
m = folium.Map(location=center, zoom_start=10)

# Protected area border
boundary_fc = ee.FeatureCollection([ee.Feature(yanachaga_geom)])
boundary_styled = boundary_fc.style(color="000000", fillColor="00000000", width=3)
boundary_mapid = boundary_styled.getMapId({})
folium.TileLayer(
    tiles=boundary_mapid["tile_fetcher"].url_format,
    attr="Protected Area",
    name="Protected Area Boundary",
    overlay=True,
    control=True
).add_to(m)

# Add styled bird layers
for species_name, asset_id in bird_layers.items():
    if asset_id:
        fc = ee.FeatureCollection(asset_id)
        color = bird_colors[species_name]
        styled = fc.style(color=color, width=2, pointSize=6)
        mapid = styled.getMapId({})

        folium.TileLayer(
            tiles=mapid["tile_fetcher"].url_format,
            attr="Earth Engine",
            name=f"{species_name}",
            overlay=True,
            control=True
        ).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Create legend with full names
legend_html = """
<div style="
  position: fixed;
  bottom: 30px;
  right: 10px;
  z-index:9999;
  background-color:white;
  padding:10px;
  border:2px solid gray;
  font-size:12px;
  box-shadow:2px 2px 6px rgba(0,0,0,0.3);
  max-height:300px;
  overflow:auto;
">
<b>Legend - Previous Bird Observations</b><br>
<div><span style="background:#e41a1c;width:12px;height:12px;display:inline-block;"></span> Andigena hypoglauca</div>
<div><span style="background:#4daf4a;width:12px;height:12px;display:inline-block;"></span> Doliornis sclateri</div>
<div><span style="background:#984ea3;width:12px;height:12px;display:inline-block;"></span> Gallinago jamesoni</div>
<div><span style="background:#ff7f00;width:12px;height:12px;display:inline-block;"></span> Hapalopsittaca melanotis</div>
<div><span style="background:#a65628;width:12px;height:12px;display:inline-block;"></span> Pionus tumultuosus</div>
<div><span style="background:#f781bf;width:12px;height:12px;display:inline-block;"></span> Pipile cumanensis</div>
<div><span style="background:#999999;width:12px;height:12px;display:inline-block;"></span> Rupicola peruvianus</div>
</div>
"""



m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
m

In [4]:
# Save map
m.save("bird_locations.html")